## **演示1201：ID3决策树的构成**

### **提出问题**
给定一批应聘者的数据，每条数据包含几个特征(level,lang,tweets,phd)以及最终是否被录用的结果(Hired)   
![](../images/120101.png)  
对于新给定的应聘这数据，如何判断其是否应该被录用？

### **分析问题**
* 决策树的基本概念
 * 决策树是通过一系列规则对数据进行分类的过程。它提供一种在什么条件下会得到什么值的类似规则的方法。决策树分为分类树和回归树两种，分类树对离散变量做决策树，回归树对连续变量做决策树
 * 样本所有特征中有一些特征在分类时起到决定性作用，决策树的构造过程就是找到这些具有决定性作用的特征，根据其决定性程度来构造一个树--决定性作用最大的那个特征作为根节点，然后递归找到各分支下子数据集中次大的决定性特征，直至子数据集中所有数据都属于同一类
 * 构造决策树的过程本质上就是根据数据特征将数据集分类的递归过程，我们需要解决的第一个问题就是，当前数据集上哪个特征在划分数据分类时起决定性作用
 * 要获取对分类结果起决定性作用的特征，可以借助信息论中的条件熵或信息增益来进行判断。对每个特征分别计算信息增益，增益最大的那个就可以视为起最决定作用的特征
* ID3：基于信息论的决策树算法
 * 算法步骤：  
    1. 以所有样本为工作数据集
    2. 分别计算每个特征的条件熵，选取条件熵最小的那个特征作为第一级主特征，并假设该特征有$n1$种取值$x_1,x_2,\cdots,x_{n1}$
    3. 在第一级主特征下设置$n1$个分支$D(D_1,D_2,\cdots,D_{n1})$，分支$D_i$的工作数据集设置为所有包含特征值$x_i$的样本；
    4. 针对每个分支$D_i$  
      1) 检查该分支下的所有工作数据集，如果每条数据的判别结果都相同，则直接以该判别结果作为叶子节点，结束该分支的构建；如果判别结果有多个，则继续执行2)  
      2) 在该分支的工作数据集中，计算除了上一级主特征之外的其它特征(称为候选特征)的条件熵，选取条件熵最小的那个特征作第二级主特征  
      3) 如果被选中的二级主特征的条件熵为0，则说明该特征已经可以完全判别样本，以该特征的取值作为分支，其取值对应的判别结果作为叶子节点，完成该分支树的构建  
      4) 如果二级主特征条件熵大于0，则根据该特征的取值，重新设置样本中包含该特征值工作数据集，开始新一轮的计算  
    5. 重复执行4，直到所有分支都确定了叶子节点，从而完成决策树的构建
 * 一旦建立了决策树，对于新的数据，可以从根节点开始逐个检索得出最终的判别结果
 * 缺点：偏向于具有大量值的属性--就是说在训练集中，某个属性所取的不同值的个数越多，那么越有可能拿它来作为分类属性，而这样做有时候是不合理的的。此外，它不能处理特征值是连续变量的情形
* ID3的改进算法
 * C4.5是ID3的一个改进算法，用信息增益率来选择属性。C4.5支持对连续特征变量的离散化处理
 * CART算法的全称是Classification And Regression Tree，采用的是Gini指数（选Gini指数最小的特征）作为分类标准。本课程不作进一步解释

### **案例1：基于ID3算法的决策树构建过程演示**
* 本例演示如何计算条件熵，选择根节点、二级节点，以及如何判断分支构建结束  
* 最终构成的树结构如下图所示：  
![](../images/120102.png)

#### **步骤1：准备样本数据**
* 每条样本数据包含某个应聘者的4个特征：level, lang, tweets, phd，并对应一个分类结果(是否录用)：True/False

In [1]:
''' 样本数据 '''

import numpy as np
import collections as col

inputs = [
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False)
]

#### **步骤2：计算特征'level'的条件熵**
* 特征level的条件熵的计算公式：  
$H(Y|X_{level})=H(Y|X_{level=Senior})*P(X_{level=Senior})+H(Y|X_{level=Mid})*P(X_{level=Mid})+H(Y|X_{level=Junior})*P(X_{level=Junior})$
* level=Senior时的条件熵计算公式：  
$H(Y|X_{level=Senior})=H(Y_{True}|X_{level=Senior})+H(Y_{False}|X_{level=Senior})$  
* level=Senior时，Y=True情形的信息熵的计算公式：  
$H(Y_{True}|X_{level=Senior})=-P(Y_{True}|X_{level=Senior})*log(P(Y_{True}|X_{level=Senior}))$

In [2]:
attribute = 'level'
groups = col.defaultdict(list)
# 整理'level'特征的每种取值('Senior', 'Mid', 'Junior')所对应的样本
for input in inputs:
    key = input[0][attribute]
    groups[key].append(input)
# 输出按特征取值进行分组的样本数据
print("按level特征取值分组的样本数据：")
for key in groups.keys():
    print('level=', key, ":")
    print(groups[key])

subsets = groups.values()
total_count = sum(len(subset) for subset in subsets)
print("样本数量：", total_count)
H_subsets = []
for subset in subsets:    # 每一个subset代表level为某个特定取值(例如level=Senior)下的所有样本
    # 计算该特定取值下，样本属于不同分类结果(Y)的概率
    labels = [label for _, label in subset]    # labels保存了该特定取值下，每个样本的的分类结果(在本例中是True/False)
    subset_total_count = len(labels)    # 该特定取值下的样本总数
    P_Y = [count / subset_total_count for count in col.Counter(labels).values()]
    # 计算特征level在该特定取值下的条件熵
    H_subset = np.sum([-p * np.log(p) for p in P_Y])
    H_subsets.append(H_subset)
# 计算特征level的条件熵
H_level = np.sum([(len(subset) / total_count) * H_subset for subset, H_subset in zip(subsets, H_subsets)])
print("level特征的条件熵：", H_level)

按level特征取值分组的样本数据：
level= Senior :
[({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'no'}, False), ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'yes'}, False), ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, False), ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True), ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'yes'}, True)]
level= Mid :
[({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True), ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, True), ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, True), ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd': 'no'}, True)]
level= Junior :
[({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True), ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True), ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, False), ({'level': 'Junior', 'lang': 'Python', 'twee

#### **步骤3： 封装计算某个特征的条件熵的函数并计算主特征**
* 将步骤2中的代码一直到一个函数中
* 计算样本中每个特征的条件熵，然后选出最小的那个特征(主要特征)

In [3]:
def conditional_entropy_by(inputs, attribute):
    groups = col.defaultdict(list)
    for input in inputs:
        key = input[0][attribute]
        groups[key].append(input)

    subsets = groups.values()
    total_count = sum(len(subset) for subset in subsets)
    H_subsets = []
    for subset in subsets:    # 每一个subset代表level为某个特定取值(例如level=Senior)下的所有样本
        # 计算该特定取值下，样本属于不同分类结果(Y)的概率
        labels = [label for _, label in subset]    # labels保存了该特定取值下，每个样本的的分类结果(在本例中是True/False)
        subset_total_count = len(labels)    # 该特定取值下的样本总数
        P_Y = [count / subset_total_count for count in col.Counter(labels).values()]
        # 计算特征level在该特定取值下的条件熵
        H_subset = np.sum([-p * np.log(p) for p in P_Y])
        H_subsets.append(H_subset)
    # 计算特征level的条件熵
    H_level = np.sum([(len(subset) / total_count) * H_subset for subset, H_subset in zip(subsets, H_subsets)])
    return H_level

print("样本中第一个关键特征的条件熵计算：")
features = ['level', 'lang', 'tweets', 'phd']   # 候选的4个特征
conditional_entropies = []
for key in features:
    conditional_entropy = conditional_entropy_by(inputs, key)
    conditional_entropies.append(conditional_entropy)
    print(key, ":", conditional_entropy)
print("关键特征(最小条件熵):", features[np.argmin(conditional_entropies)])

样本中第一个关键特征的条件熵计算：
level : 0.48072261929232607
lang : 0.5961978716754623
tweets : 0.5465122114944403
phd : 0.6183974457364384
关键特征(最小条件熵): level


#### **步骤4：计算第二级条件熵**
* 选择前面计算的特征条件熵最小的那个特征，作为第一级主特征；落选的特征形成新的候选特征数组
* 将第一级主特征中的每个取值分别设置为一个分支(例如：level=Senior是第1个分支，level=Mide是第2个分支，level=Junior是第3个分支)。每个分支中包含了样本数据中包含该特征取值的那些样本(例如，第1个分支中包含了样本数据中所有level=Senior的那些样本)
* 在每个分支中计算候选特征数组中每个特征的条件熵，选择条件熵最小的那个作为第二级主特征

In [5]:
# 取出第一级主特征，剩余'lang','tweets','phd'三个候选特征
features =  ['lang', 'tweets', 'phd']
conditional_entropies = []
# 分支level=Senior下，各候选特征条件熵计算
senior_inputs = [(input, label) for input, label in inputs if input["level"] == "Senior"]
print("(2.1)分支level=Senior下的条件熵计算：")
for key in features: 
    conditional_entropy = conditional_entropy_by(senior_inputs, key)
    conditional_entropies.append(conditional_entropy)
    print(key, ":", conditional_entropy)
print("条件熵最小的特征:", features[np.argmin(conditional_entropies)])
# lang : 0.277258872224
# tweets : 0.0
# phd : 0.659167373201
# 特征tweets具有最小的条件熵，因此以tweets为Senior分支的二级节点，其两个取值('yes','no')分别作为两个分支
# 同时，因为tweets的条件熵已经为0，说明该特征已经完全可以判别，因此无需下级节点
# tweets节点下取值为'yes'的分支对应着最终判别结果True，取值为'no'的分支对应着最终判别结果'False'

conditional_entropies = []
# 分支level=Junior下，各候选特征条件熵计算
junior_inputs = [(input, label) for input, label in inputs if input["level"] == "Junior"]
print("(2.2)分支level=Junior下的条件熵计算：")
for key in features: 
    conditional_entropy = conditional_entropy_by(junior_inputs, key)
    conditional_entropies.append(conditional_entropy)
    print(key, ":", conditional_entropy)
print("条件熵最小的特征:", features[np.argmin(conditional_entropies)])
# lang : 0.659167373201
# tweets : 0.659167373201
# phd : 0.0
# 特征phd具有最小的条件熵，因此以phd为Junior分支的二级节点，其两个取值('yes','no')分别作为两个分支
# 同时，因为phd的条件熵已经为0，说明该特征已经完全可以判别，因此无需下级节点
# phd节点下取值为'yes'的分支对应着最终判别结果False，取值为'no'的分支对应着最终判别结果True

conditional_entropies = []
# 分支level=Mid下，各候选特征条件熵计算
mid_inputs = [(input, label) for input, label in inputs if input["level"] == "Mid"]
print("(2.3)分支level=Mid下的条件熵计算：")
for key in features: 
    conditional_entropy = conditional_entropy_by(mid_inputs, key)
    conditional_entropies.append(conditional_entropy)
    print(key, ":", conditional_entropy)
# lang:0.0  tweets:0.0  phd:0.0
# 事实上，level='mid'时，所有训练样本都被归类为True，因此它本身已经完全可以作为最终判断节点，无需再对其它特征进行下一步计算

(2.1)分支level=Senior下的条件熵计算：
lang : 0.2772588722239781
tweets : 0.0
phd : 0.6591673732008658
条件熵最小的特征: tweets
(2.2)分支level=Junior下的条件熵计算：
lang : 0.6591673732008658
tweets : 0.6591673732008658
phd : 0.0
条件熵最小的特征: phd
(2.3)分支level=Mid下的条件熵计算：
lang : 0.0
tweets : 0.0
phd : 0.0


#### **步骤5：判断该特征是否叶子节点，或者继续递归**
* 在上一轮计算中，对于每一个分支，分别检查每个候选特征的条件熵
* 如果某个候选特征的条件熵为0，则说明该特征已经足以完全确定分类结果，从而该特征作为叶子特征，不必再向下计算；该分支结束
* 若该分支下所有候选特征的条件熵均不为0，则选择最小的那个，作为第三级主特征，继续进后续计算